In [2]:
import json
import os
import re
import nltk
import time
import pickle
from nltk import pos_tag
from six import string_types

from nltk.tokenize import TweetTokenizer

from nltk.corpus.reader.util import StreamBackedCorpusView, concat, ZipFilePathPointer
from nltk.corpus.reader.api import CorpusReader

DOC_PATTERN = r'.*\.json' 
# PKL_PATTERN = r'.*\.pickle'

##########################################################################
## Tweets Corpus Reader
##########################################################################

class TweetsCorpusReader(CorpusReader):

    """
    Reader for corpora that consist of Tweets represented as a list of line-delimited JSON.
    Individual Tweets can be tokenized using the default tokenizer, or by a
    custom tokenizer specified as a parameter to the constructor.
    Construct a new Tweet corpus reader for a set of documents
    located at the given root directory.
    If you want to work directly with the raw Tweets, the `json` library can
    be used::
       import json
       for tweet in reader.docs():
           print(json.dumps(tweet, indent=1, sort_keys=True))
    """

    CorpusView = StreamBackedCorpusView  # The corpus view class used by this reader.

    def __init__(self, root, fileids=None, word_tokenizer=TweetTokenizer(), encoding='utf-8-sig', bullying_trace='bullying_trace'):

        CorpusReader.__init__(self, root, fileids, encoding, bullying_trace)

        for path in self.abspaths(self._fileids):
            if isinstance(path, ZipFilePathPointer):
                pass
            elif os.path.getsize(path) == 0:
                raise ValueError("File {} is empty".format(path))
        """Check that all user-created corpus files are non-empty."""

        self._word_tokenizer = word_tokenizer
        self._bullying_trace = bullying_trace
    
    def _read_tweets(self, stream):
        """
        Assumes that each line in ``stream`` is a JSON-serialised object.
        """
        tweets = []
        for i in range(10):
            line = stream.readline()
            if not line:
                return tweets
            tweet = json.loads(line)
            tweets.append(tweet)
        return tweets

    def docs(self, fileids=None, bullying_trace=None):
        """
        Returns the full Tweet objects, 
        :return: the given file(s) as a list of dictionaries deserialised from JSON.
        :rtype: list(dict)
        """
        tweets = concat(
            [
                self.CorpusView(path, self._read_tweets, encoding=enc)
                for (path, enc, fileid) in self.abspaths(fileids, True, True)
            ])
        
        tweets = [tweet for tweet in tweets]

        if self._bullying_trace=='bullying_trace':
            remove = ['nan','remove']
            tweets = [tweet for tweet in tweets if str(tweet[self._bullying_trace]) not in remove]
        if self._bullying_trace=='bullying_role':
            tweets = [tweet for tweet in tweets if str(tweet[self._bullying_trace]) !='nan']
        elif self._bullying_trace=='form_of_bullying':
            tweets = [tweet for tweet in tweets if str(tweet[self._bullying_trace]) !='nan']
        elif self._bullying_trace=='bullying_post_type':
            tweets = [tweet for tweet in tweets if str(tweet[self._bullying_trace]) !='nan']           
        else:
            tweets

        return tweets 
        
    # def docs(self, fileids=None, bullying_trace=None):
    #     """
    #     Returns the full Tweet objects, 
    #     :return: the given file(s) as a list of dictionaries deserialised from JSON.
    #     :rtype: list(dict)
    #     """
    #     tweets = concat(
    #         [
    #             self.CorpusView(path, self._read_tweets, encoding=enc)
    #             for (path, enc, fileid) in self.abspaths(fileids, True, True)
    #         ])
        
    #     # if self._bullying_trace=='bullying_trace':
    #     #     new_tweets = [tweet for tweet in tweets if str(tweet[self._bullying_trace]) =='yes']
    #     if self._bullying_trace=='bullying_role':
    #         new_tweets = [tweet for tweet in tweets if str(tweet[self._bullying_trace]) !='nan']
    #     elif self._bullying_trace=='form_of_bullying':
    #         new_tweets = [tweet for tweet in tweets if str(tweet[self._bullying_trace]) !='nan']
    #     elif self._bullying_trace=='bullying_post_type':
    #         new_tweets = [tweet for tweet in tweets if str(tweet[self._bullying_trace]) !='nan']           
        
    #     else:
    #         remove = ['nan','remove']
    #         new_tweets = [tweet for tweet in tweets if str(tweet[self._bullying_trace]) not in remove]

    #     return new_tweets
    #     # return tweets 
           
    def sizes(self, fileids=None):
        """
        Returns a list of tuples, the fileid and size on disk of the file.
        This function is used to detect oddly large files in the corpus.
        """
        # Create a generator, getting every path and computing filesize
        for path in self.abspaths(fileids):
            yield os.path.getsize(path)

    def fields(self, fields, fileids=None):
        """
        extract particular fields from the json doc. Can be string or an 
        iterable of fields. If just one fields in passed in, then the values 
        are returned, otherwise dictionaries of the requested fields returned
        """
        if isinstance(fields, string_types):
            fields = [fields,]

        if len(fields) == 1:
            for doc in self.docs(fileids):
                if fields[0] in doc:
                    yield doc[fields[0]]

        else:
            for doc in self.docs(fileids):
                yield {
                    key : doc.get(key, None)
                    for key in fields
                }
    
    def strings(self, fileids=None):
        """
        Returns only the text content of Tweets in the file(s)
        :return: the given file(s) as a list of Tweets.
        :rtype: list(str)
        """
        fulltweets = self.docs(fileids)
        tweets = []
        for jsono in fulltweets:
            try:
                text = jsono['full_tweet']
                if isinstance(text, bytes):
                    text = text.encode('latin-1').decode('utf-8') #.decode(self.encoding) #
                tweets.append(text)
            except KeyError:
                pass
        return tweets

    def tokenized(self, fileids=None):
        """
        :return: the given file(s) as a list of the text content of Tweets as
        as a list of words, screenanames, hashtags, URLs and punctuation symbols.
        :rtype: list(list(str))
        """
        tweets = self.strings(fileids)
        tokenizer = self._word_tokenizer
        tokenized = [tokenizer.tokenize(t) for t in tweets]
        return [ pos_tag(token) for token in tokenized ]
    
    def process_tweet(self, fileids=None):

		# emoji_pattern = re.compile("["
        # u"\U0001F600-\U0001F64F"  # emoticons
        # u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        # u"\U0001F680-\U0001F6FF"  # transport & map symbols
        # u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        # u"\U00002702-\U000027B0"
        # u"\U000024C2-\U0001F251"
        # "]+", flags=re.UNICODE)
        tagged_tweets = self.tokenized(fileids)
        mod_tweets = []
        for tweet in tagged_tweets:
            mod_tweet=[]
            for (token, tag) in tweet:    
                if '@' in token:
                    token = '@user'
                elif '#' in token:
                    token = token[1:]
                elif ('http' or 'https') in token:
                    token = 'UR'
                elif token == 'luv':
                    token = 'love'
                else:
                    pass
                mod_tweet.append((token,tag))
            mod_tweets.append(mod_tweet)
        return mod_tweets

    def raw(self, fileids=None):
        """
        Return the corpora in their raw form.
        """
        if fileids is None:
            fileids = self._fileids
        elif isinstance(fileids, string_types):
            fileids = [fileids]
        return concat([self.open(f).read() for f in fileids])

    def describe(self, fileids=None):
        """
        Performs a single pass of the corpus and
        returns a dictionary with a variety of metrics
        concerning the state of the corpus.
        """
        started = time.time()
        # Structures to perform counting.
        counts  = nltk.FreqDist()
        tokens  = nltk.FreqDist()

        # Compute the number of files and categories in the corpus
        n_fileids = len(self.fileids())

        # Perform single pass over paragraphs, tokenize and count
        for tweet in self.tokenized(fileids):
            counts['tweets'] += 1

            for word in tweet:
                counts['words'] += 1
                tokens[word] += 1
            # print(counts, tokens)

        # Return data structure with information
        return {
            'files':  n_fileids,
            'tweets':  counts['tweets'],
            'words':  counts['words'],
            'vocab':  len(tokens),
            'lexdiv': float(counts['words']) / float(len(tokens)),
            'twdoc':  float(counts['tweets']) / float(n_fileids),
            'mins':   (time.time() - started)/60,
        }
    
    def describes(self, fileids=None, categories=None):
        """
        Returns a string representation of the describe command.
        """
        return (
            "This twitter corpus contains {files} files.\n"
            "Structured as:\n"
            "    {tweets} tweets ({twdoc:0.3f} mean tweets per file)\n"
            "    Word count of {words} with a vocabulary of {vocab}\n"
            "    ({lexdiv:0.3f} lexical diversity).\n"
            "Corpus scan took {mins:0.3f} minutes."
        ).format(**self.describe(fileids))

In [4]:
from pathlib import Path
import pandas as pd
import os


# ROOT = Path('C:\\Users\\niti.mishra\\Documents\\Personal\\cyberbullying\\') # windows
ROOT = Path('/Users/peaceforlives/Documents/Projects/cyberbullying/')         # mac
CORPUS = Path.joinpath(ROOT, 'data', 'labelled_tweets', 'ab')
DOC_PATTERN = r'.*\.json' 
target = 'bullying_trace'

corpus = TweetsCorpusReader(CORPUS.__str__(), DOC_PATTERN, bullying_trace=target)        
X = pd.DataFrame(corpus.strings())
X['y'] = list(corpus.fields(target))
pd.options.display.max_colwidth = 5000
X.head()

,0,y
0,"@heyyyitsbieha bullying and helping is two different things,i didn’t force him,im not using the rock’s voice or the big show’s slam,i’m just asking him if he’s free and would like to help,but i’m agreeing with you that bullying is a trash behavior!✌🏻",yes
1,@umineko_lover @mienumirai i mean it has some really graphic depictions of bullying and is pretty depressing at times but the ending (of the first game at least) is good and has that message.,no
2,my take on social issues like this is that everything sucks because everything sucks.\n\nvictim was probably bullied into submission directly and indirectly. perpetrator did it because he wanted to be grown up so fast because his family situation sucked which-,yes
3,"i want to watch a group of characters ranging in the spectrum of kind-hearted to decent enough to tolerate be goofy, funnily stupid or god for once witty. not a bunch of bitches bully each other with a laugh track in the background.",no
4,i’d never let a fucking fetus on twitter bully me into deactivating got me fucked up. bitch i’ll literally shove u back into ur mother’s womb,no


In [10]:
X.shape

(7868, 2)

In [6]:
import glob 
from pathlib import Path
import pandas as pd
import os

import matplotlib.pyplot as plt

In [7]:
PATH = r'/Users/peaceforlives/Documents/Projects/cyberbullying/final_classification/results/'
data = pd.read_pickle(PATH+'data_docs.pkl')
data.drop('source', axis=1, inplace=True)
data.loc[data['bullying_trace']=='no', ['bullying_role', 'form_of_bullying', 'bullying_post_type']] = 'NA'
data.shape

(839680, 14)

In [8]:
data['bullying_trace'].value_counts()

no     552600
yes    287080
Name: bullying_trace, dtype: int64

In [9]:
cols = ['full_tweet', 'bullying_trace', 'bullying_role', 'form_of_bullying', 'bullying_post_type']
df = data[cols]
df = df.sample(frac=1)
pd.options.display.max_colwidth = 5000
df.head(50)

,full_tweet,bullying_trace,bullying_role,form_of_bullying,bullying_post_type
394955,"@niksaf0046 this is very true, but then again being ignored or shunned can be a form of bullying, but that is getting into semantics. ^0^\n\nhey maybe we can cancel each other out with this. i'll write something for your art and you can create art for something i write and we can be even! ^-^ https://t.co/fm5usfhdlz",no,NA,NA,NA
758388,"day 127\nwonho is a victim of targeted cyberbullying, defamation and baseless accusations unproven to this day. he's constantly being deleted as he never existed.\nkeep saying his name, keep being loud about this.\nhoseok we are here.\n@officialmonstax \n@starshipent\n#noendformonstax7 https://t.co/pmnwtdekav",yes,defender,cyber,report
589972,"@dizzykook lily and she was a bully, she was always hitting me and i never leave her side because she thought me that she was my only friend and that’s what friends do😷",yes,victim,general,self-disclosure
406532,@swamptalking @ranchomancho @notgoingtohelp @lakelife706 @dykeules then stop being such a pussy \n\nand don't pretend to be kind because you're not kind\n\nyou're pretend kind for clut lol\n\ni bet you've never helped a single soul in real life\n\njust act like a cry bully on twitter,no,NA,NA,NA
420648,@eddyburback i mean it was bullying straight up and thats always been a part of the tos so,no,NA,NA,NA
553835,via @hotairblog: self-cancellation: author asked magazine to remove story after being bullied online https://t.co/comasfvquc https://t.co/zf38luwmdg,yes,reporter,cyber,report
102909,"*when notre dame was burning*\nmedia: 😒😞😔😟😕🙁☹️😣😖😫😩🥺😭😢😤😠😡🤬🤯😳😱\n*when amazon rainforest is burning, suicide occurs, school shootings, bullying, plastic eroding to the sea, discrimination*\nmedia:\n#worldsuicidepreventionday https://t.co/aci2i517gg",no,NA,NA,NA
825754,@ixdavidsb i want to say well i'm not sure if my friend has twitter but from instagram helped me get past through depression and cyberbully in other game community which was my first community that i went but there's a back story and me which is depressing for me and i wouldn't be....1/2,yes,victim,cyber,self-disclosure
100749,i am a survivor and was mercilessly bullied by my abuser &amp; his friends when i would make noise about him being unsafe. i know what it is like for people to try to force you back into hiding.,yes,victim,general,self-disclosure
456276,asim is the only contestant who has been targeted and bullied and mocked so badly yet he remained so strong he has been physically assaulted bruised up everything he comes out stronger everytime he is a sher he has so much positivity in him #foreverwithasim,yes,reporter,general,report


In [8]:
b_trace = data[data['bullying_trace']=='yes']
b_trace.shape

(287080, 14)

In [9]:
b_trace.head()

,created_at,id,geo,coordinates,place,lang,full_tweet,bullying_trace,bullying_role,form_of_bullying,bullying_post_type,timezone,localtime,state
0,2019-08-07 14:13:45+00:00,1159105350747901953,None,None,None,en,"i mean most @ my school are fine, but the kpop...",yes,victim,general,self-disclosure,None,None,None
3,2019-08-07 14:14:47+00:00,1159105611684089856,None,None,None,en,@jayb_1411 that’s why i hate that people now c...,yes,victim,general,self-disclosure,None,None,None
9,2019-08-07 14:18:23+00:00,1159106516844044291,None,None,None,en,@namaslays apparently claims of bullying a cre...,yes,reporter,cyber,report,None,None,None
12,2019-08-07 14:30:40+00:00,1159109609241665536,None,None,None,en,@yslattess i am being falsely targeted and acc...,yes,victim,general,self-disclosure,None,None,None
13,2019-08-07 14:30:50+00:00,1159109651113414657,None,None,None,en,okay that fuck # mjfam bullshit is q fucking b...,yes,defender,cyber,report,None,None,None


In [63]:
for column in b_trace.iloc[:,8:11]:
    print(b_trace[column].value_counts())

victim      113486
reporter     91116
defender     62654
accuser      16197
bully         3171
other          456
Name: bullying_role, dtype: int64
general     195379
cyber        90958
physical       657
verbal          86
Name: form_of_bullying, dtype: int64
report             117739
self-disclosure    102315
accusation          67025
denial                  1
Name: bullying_post_type, dtype: int64


In [72]:
pd.options.display.max_colwidth = 50
test = b_trace[b_trace['bullying_role']=='bully']
# test = b_trace[~b_trace['full_tweet'].str.match('love')]
# test.iloc[:,6]
test.shape

(3171, 14)

In [85]:
geo_data = data[data['timezone'].notnull()]
geo_data.shape

(268, 14)

In [88]:
pd.options.display.max_colwidth = 1000
geo_data.drop(['geo','coordinates','place','lang'], axis=1, inplace=True)
geo_data

//anaconda3/envs/twitter_cyberbullying/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,created_at,id,full_tweet,bullying_trace,bullying_role,form_of_bullying,bullying_post_type,timezone,localtime,state
438,2019-08-07 16:41:16+00:00,1159142476520673281,"devote your energy to something greater than yourself. you’ll undoubtedly be hated for it. you’ll be bullied, intimidated, called every name in the book and maybe even killed. but truth lasts and illusions fade. do… https://t.co/httiqpeimd",no,reporter,general,report,America/Los_Angeles,2019-08-07 09:41:16 PDT-0700,Oregon
1400,2019-08-07 21:54:53+00:00,1159221398172102656,https://t.co/dcgfsfxogy operation #dropoutjeep \nthey can control/intercept/reroute apple products communication like iphones and easily destroy lives with their #cyberbullying… https://t.co/hequxeqxus,no,accuser,cyber,report,America/Detroit,2019-08-07 17:54:53 EDT-0400,Michigan
1702,2019-08-07 23:38:51+00:00,1159247564081979392,performing live at @biglumusic \notg_fashawn bout to have the ground shaking! hit him up for tickets. all age event 🎯🔥🔥🔥\notg🖤\n3632 nw 25th st 33142\n(club bully) 10pm - 3am\nladies in free all night 💰\n•\n•\n@biglumusic… https://t.co/iylncn2lgm,no,reporter,general,report,America/New_York,2019-08-07 19:38:51 EDT-0400,Florida
3099,2019-08-08 12:02:41+00:00,1159434756318269441,"✔🆓️ parent workshop sat 8/10 - learn about online privacy, prevent cyberbullying and more... register: https://t.co/2a5yalypsn \ngetting ready for #backtoschool 🏫 ? @cityofdoral comcast are sponsoring… https://t.co/gqdbusbozf",no,reporter,cyber,report,America/New_York,2019-08-08 08:02:41 EDT-0400,Florida
4071,2019-08-08 20:05:03+00:00,1159556148153864192,#repost @mjcsolutions\n• • • • •\nthis offer is exclusive for you! \nonly $9.99 link in bio\n\n✔are you dealing with a toxic boss?\n✔is your boss a bully? \n✔are you the bully? \n\nenroll in this quick 28 minutes online… https://t.co/mjy3evjczd,no,defender,cyber,accusation,America/Los_Angeles,2019-08-08 13:05:03 PDT-0700,California
...,...,...,...,...,...,...,...,...,...,...
795255,2020-03-16 20:34:28+00:00,1239651266348843008,"don’t stand for republican bullying!!!! they still haven’t forced companies to stop making us come in for work, then they can’t force us not to get out and vote!!!! this is a republican ploy to stop us from kicking… https://t.co/w37vdbxifo",no,defender,general,report,America/New_York,2020-03-16 16:34:28 EDT-0400,Ohio
800017,2020-03-18 16:39:22+00:00,1240316877705891841,"it is with great sorrow that we announce we will temporary close until the current situation has receded.\n\nwe have tried to remain open in a safe manner however have had several people bully us online and in person,… https://t.co/wjq7y60m98",no,defender,cyber,report,America/Los_Angeles,2020-03-18 09:39:22 PDT-0700,California
824303,2020-03-27 14:13:56+00:00,1243541770924892162,https://t.co/5fjvchuuok has links to every movie i'm in that's available to watch online. this pic is from bully! just released this year! will you watch it pls? tell me what you think? it's available on basically every… https://t.co/jabj375neg,yes,victim,cyber,report,America/New_York,2020-03-27 10:13:56 EDT-0400,New Jersey
834979,2020-03-30 21:00:15+00:00,1244731185164480512,this is what my day has been. \nwho in the actual heck gave this girl the right to &amp; bully me through private messages on instagram &amp; facebook? i've been bullied my whole entire life &amp; words can hurt &amp; no one will… https://t.co/4vxxrc9cpy,yes,victim,general,self-disclosure,America/Chicago,2020-03-30 16:00:15 CDT-0500,Florida


In [106]:
geo_data['state'].value_counts().to_clipboard()

In [109]:
geo_data.groupby('state')['bullying_trace'].count()

state
Alabama              5
Alaska               1
Alberta              3
Arizona              4
British Columbia     2
California          44
Colorado             4
Connecticut          6
Delaware             2
Florida             19
Georgia              6
Illinois             4
Kansas               1
Kentucky             1
Louisiana            6
Manitoba             2
Maryland             5
Massachusetts        2
Michigan            11
Minnesota            1
Missouri             3
Montana              1
Nebraska             1
Nevada               4
New Jersey           3
New York            12
North Carolina       4
North Dakota         2
Nova Scotia          2
Ohio                 4
Ontario             22
Oregon               4
Pennsylvania        12
Québec              19
Saskatchewan         1
South Carolina       1
Tennessee            7
Texas               21
Utah                 2
Virginia             5
Washington           4
West Virginia        3
Wisconsin            2
Name:

In [118]:
geo_bully = geo_data[geo_data['bullying_trace']=='yes']
geo_bully.shape

(60, 10)

In [119]:
time = geo_bully.copy()
time = time[['bullying_trace','bullying_role','form_of_bullying','bullying_post_type','timezone','localtime']]
time.head()

,bullying_trace,bullying_role,form_of_bullying,bullying_post_type,timezone,localtime
16546,yes,victim,cyber,self-disclosure,America/Toronto,2019-08-13 06:29:41 EDT-0400
39524,yes,reporter,general,report,America/New_York,2019-08-21 09:56:26 EDT-0400
48072,yes,victim,general,self-disclosure,America/Los_Angeles,2019-08-23 18:12:49 PDT-0700
80390,yes,victim,general,self-disclosure,America/Los_Angeles,2019-09-03 10:48:19 PDT-0700
107754,yes,reporter,general,report,America/New_York,2019-09-11 18:19:16 EDT-0400


In [124]:
time['time'] = pd.to_datetime(time['localtime']) #, format='%d%b%Y:%H:%M:%S.%f')
time.head()

,bullying_trace,bullying_role,form_of_bullying,bullying_post_type,timezone,localtime,time
16546,yes,victim,cyber,self-disclosure,America/Toronto,2019-08-13 06:29:41 EDT-0400,2019-08-13 06:29:41-04:00
39524,yes,reporter,general,report,America/New_York,2019-08-21 09:56:26 EDT-0400,2019-08-21 09:56:26-04:00
48072,yes,victim,general,self-disclosure,America/Los_Angeles,2019-08-23 18:12:49 PDT-0700,2019-08-23 18:12:49+07:00
80390,yes,victim,general,self-disclosure,America/Los_Angeles,2019-09-03 10:48:19 PDT-0700,2019-09-03 10:48:19+07:00
107754,yes,reporter,general,report,America/New_York,2019-09-11 18:19:16 EDT-0400,2019-09-11 18:19:16-04:00


In [139]:
time['time'].value_counts().sort_index()